# Title
Description of the notebook

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Imports
import os, json, requests
from requests.auth import HTTPBasicAuth
from pprint import pprint

import time
import pdb

In [ ]:
# Setup Planet Data API base URL
BASE_URL = "https://api.planet.com/data/v1"
STATS_URL = f'{BASE_URL}/stats'


# Planet data fetch defaults
item_type = 'PSScene4Band'

## Planet api client

In [5]:
from planet import api

In [6]:
print(client.auth.value)

7d8af35b6e944f33bb5e33ada32ab4a0


In [37]:
# my_key = '81c0dedd35fd4a398da77753b90d62ea' # songcoco15
# my_key = '7d8af35b6e944f33bb5e33ada32ab4a0' #hayleysong15

# Set up the Planet API client - method1
os.environ['PL_API_KEY']  = my_key
client = api.ClientV1()

# Method2: alternatively, 
# client = api.ClientV1(api_key=my_key)

In [38]:
print(client.auth.value)

81c0dedd35fd4a398da77753b90d62ea


In [3]:
## Set Area of Interest
aoi_data = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -118.2967472076416,
              34.00578318672437
            ],
            [
              -118.22739601135252,
              34.00578318672437
            ],
            [
              -118.22739601135252,
              34.07846940942791
            ],
            [
              -118.2967472076416,
              34.07846940942791
            ],
            [
              -118.2967472076416,
              34.00578318672437
            ]
          ]
        ]
      }
    }
  ]
}
aoi_geom = aoi_data['features'][0]['geometry']

## Seach scenes with filters
- `quick_search` member function
- requires filters (`planet.api.filters`


In [7]:
from planet.api import filters
from datetime import datetime

In [8]:
# geometry filter
geom_filter = filters.geom_filter(aoi_geom)

# date filter
start_date = datetime(year=2016, month=1, day=1)
end_date = datetime(year=2019, month=4, day=1)
date_filter = filters.date_range('acquired', gte=start_date, lte=end_date)

# cloud filter
cloud_filter = filters.range_filter('cloud_cover', lte=0.1)

# and filter
and_filter = filters.and_filter(
    geom_filter, date_filter, cloud_filter
)

In [9]:
print(type(and_filter))
pprint(and_filter)

<class 'dict'>
{'config': ({'config': {'coordinates': [[[-118.2967472076416,
                                          34.00578318672437],
                                         [-118.22739601135252,
                                          34.00578318672437],
                                         [-118.22739601135252,
                                          34.07846940942791],
                                         [-118.2967472076416,
                                          34.07846940942791],
                                         [-118.2967472076416,
                                          34.00578318672437]]],
                        'type': 'Polygon'},
             'field_name': 'geometry',
             'type': 'GeometryFilter'},
            {'config': {'gte': '2016-01-01T00:00:00Z',
                        'lte': '2019-04-01T00:00:00Z'},
             'field_name': 'acquired',
             'type': 'DateRangeFilter'},
            {'config': {'lte': 0.1},
          

In [10]:
and_filter.keys()

dict_keys(['type', 'config'])

- Send a HTTP request for search. Needs to specify:
    - ::filters:: for scene searches
    - item_type (eg. "PSScene4Band")

In [11]:
item_types = ['PSScene4Band']
search_req = filters.build_search_request(and_filter, item_types)
pprint(search_req)

{'filter': {'config': ({'config': {'coordinates': [[[-118.2967472076416,
                                                     34.00578318672437],
                                                    [-118.22739601135252,
                                                     34.00578318672437],
                                                    [-118.22739601135252,
                                                     34.07846940942791],
                                                    [-118.2967472076416,
                                                     34.07846940942791],
                                                    [-118.2967472076416,
                                                     34.00578318672437]]],
                                   'type': 'Polygon'},
                        'field_name': 'geometry',
                        'type': 'GeometryFilter'},
                       {'config': {'gte': '2016-01-01T00:00:00Z',
                                   'lte': '2

In [12]:
res = client.quick_search(search_req)

`items_iter` method requires the number of items to be iterated
we can get the number of items ahead of time
actually....nah, not easily (ie. not following through all the '_next' links if more than 250 items exist 
in the search result

`n_items = len(res.get()[res.ITEM_KEY])`

In [13]:
items = []
for item in res.items_iter(1000):
#     pdb.set_trace()
#     print('scene id: ', item['id'])
#     pprint(item['properties'])
    items.append(item)

In [217]:
# with open('output/results.json', 'w') as f:
#     res.json_encode(f,limit=100)

- To download the image of the scene, further specify:
    - asset_type (eg. 'analytic', 'visual')
    - needs to activate the asset first 
    - once activation succeeds, download via the download_link
    